In [1]:
import pandas as pd
import geopandas as gpd

### PREDIAL

In [2]:
casas=gpd.read_file(r'C:\Users\dlara\Naucalpan_new_casas.shp')
# terrenos=gpd.read_file(r'C:\Users\dlara\Downloads\naucalpancompleto\new_terreno1.shp')

In [3]:
predial=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIAL (16022022).xlsx')
adeudo=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIALADEUDO (16022022).xlsx')
propietario=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PROPIETARIO (16022022).xlsx')

In [4]:
base_pred= pd.read_csv(r'C:\Users\dlara\Downloads\Predial.csv')

In [9]:
from geoloc2.Valor_Catastral.calculo_predial import *

ImportError: attempted relative import with no known parent package

In [ ]:
curts= gpd.read_file(r'C:\Users\dlara\Downloads\098 Naucalpan\NAUCALPAN_PREDIOS.shp')

In [ ]:
NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')

C:\Users\dlara\AppData\Local\Temp\ipykernel_14656\1341984383.py:1: DtypeWarning: Columns (2,4,5,6,8,9,10,11,12,13,14,15,16,17,19,20,21,23,24,25,26,27,30,32,34,35,38,39,40,43,44,49,50,51,52,53,55,56,57,60,61,62,63,64,65,66,67,68,69,70,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,96,97,98,99,101,102,103,104,107,108,109,110,111,113,114,115,116,117,118,119,124,127,129,130,131,134,135,137,138,139,140,141,142,146,156,157,158,159,160,161,162,164,166,167,168,169,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,219,220) have mixed types. Specify dtype option on import or set low_memory=False.
  NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')


In [ ]:
def transform_df_to_gpd(df: pd.DataFrame,
                    lon_col: str = 'INTFIS_RS_FD_LON_1',
                    lat_col: str = 'INTFIS_RS_FD_LAT_1', crs= 'EPSG:4326'):
        '''Transforma base a partir de dos columnas de latitud y longitud'''
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(
                df[lon_col], df[lat_col]
            ),
            crs=crs,
        )
        return gdf

In [ ]:
NAUC['INTFIS_RS_FD_LAT']= NAUC['INTFIS_RS_FD_LAT_1'].combine_first(NAUC['INTFIS_RS_FD_LAT'])
NAUC['INTFIS_RS_FD_LON']= NAUC['INTFIS_RS_FD_LON_1'].combine_first(NAUC['INTFIS_RS_FD_LON'])

In [ ]:
NAUC=transform_df_to_gpd(NAUC, lon_col= 'INTFIS_RS_FD_LON',lat_col = 'INTFIS_RS_FD_LAT')

In [ ]:
PRED= NAUC.loc[NAUC['CLAVECATASTRAL'].notna()]

In [ ]:
Manzanas=gpd.read_file(r'C:\Users\dlara\Downloads\Full manzanas\Full manzanas\Manzana_Naucalpan.shp')

In [ ]:
Manzanas['geometry']=Manzanas.buffer(100)

In [ ]:
Manz=Manzanas.dissolve()

In [ ]:
NAUC=NAUC.to_crs(Manz.crs)
def new_func(NAUC):
    return gpd.sjoin(NAUC,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

rectificado_nauc=new_func(NAUC)

In [ ]:
curts=curts.to_crs(Manz.crs)
curts_rectificado=gpd.sjoin(curts,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

In [ ]:
casas=casas.to_crs(Manz.crs)
casas['ID']=casas.reset_index()['index']
casas_rectificado=gpd.sjoin(casas,Manz).drop(columns=[	'index_right',	'mun',	'zona',	'manz',	'cve_cat'])

## Cruces

In [2]:
PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')

C:\Users\dlara\AppData\Local\Temp\ipykernel_32896\94833481.py:1: DtypeWarning: Columns (47,48,54) have mixed types. Specify dtype option on import or set low_memory=False.
  PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')


In [7]:
PRED.CLAVECATASTRAL.value_counts()

02401271382C0004     16740
02401271041A0004     16368
240127102010026.0    12719
02401357010F0012     12032
0240127103220019     11904
                     ...  
240144807000000.0        1
240144809000000.0        1
240144808000000.0        1
240144806000000.0        1
240330107000000.0        1
Name: CLAVECATASTRAL, Length: 10151, dtype: int64

In [8]:
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)

In [20]:
# PRED.drop_duplicates('CLAVECATASTRAL')
PRED[['CLAVECATASTRAL', 'LONGITUD','LATITUD','CURT_1', 'curt','geometry','min_dist']].dropna()
from preparacion_datos.src.preparacion_inter_puntos import prep
# prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857).to_crs(4326)


,Unnamed: 0,index_left,CLAVE_PREDIO,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTER_1,VALORCONSTPROP,...,LONGITUD,CLAVECATASTRAL,ESTIMADO,CURT_1,geometry,LONGITUD_1,LATITUD_1,ORDEN,min_dist,0
0,0,0.0,241101898.0,241101800.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-99.759474,0241101898000000,17.0,0.0,POINT (-99.76296 19.78527),-1.110556e+07,2.247611e+06,1.0,334.266232,NaN
1,1,0.0,241101898.0,241101800.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-99.759474,0241101898000000,17.0,0.0,POINT (-99.76284 19.78245),-1.110555e+07,2.247277e+06,2.0,333.541286,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0000000000000nan,NaN,NaN,POINT (-99.76214 19.78799),-1.110547e+07,2.247933e+06,3.0,330.736075,NaN
3,3,0.0,241101898.0,241101800.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-99.759474,0241101898000000,17.0,0.0,POINT (-99.76158 19.77989),-1.110541e+07,2.246975e+06,4.0,333.541286,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0000000000000nan,NaN,NaN,POINT (-99.76012 19.79004),-1.110525e+07,2.248175e+06,5.0,226.941789,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885218,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0000000000000nan,NaN,NaN,POINT (-99.86125 19.80562),-1.111650e+07,2.250018e+06,71.0,161.477615,NaN
1885219,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0000000000000nan,NaN,NaN,POINT (-99.86107 19.80697),-1.111648e+07,2.250178e+06,72.0,161.492671,NaN
1885220,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0000000000000nan,NaN,NaN,POINT (-99.84826 19.82712),-1.111506e+07,2.252562e+06,1.0,1243.804891,NaN
1885221,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0000000000000nan,NaN,NaN,POINT (-99.82294 19.84776),-1.111224e+07,2.255005e+06,4.0,1243.804891,NaN


In [5]:
PRED=PRED.dropna(axis=1, how='all')

In [6]:
PRED

,Unnamed: 0,index_left,CLAVE_PREDIO,CLAVE_CATASTRAL,CLAVE_MANZ,fol_rec,nombre_1,rfc_1,ubicacion,domicilio_fiscal,...,LATITUD,LONGITUD,CLAVECATASTRAL,ESTIMADO,CURT_1,geometry,LONGITUD_1,LATITUD_1,ORDEN,min_dist
0,0,0.0,241101898.0,241101800.0,241101800.0,IT759157,GABRIEL T. VICENTE*,XAXX010101000,JOSE MARIA MORELOS SANTIAGO ACUTZILAPAN,NaN,...,19.779456,-99.759474,0241101898000000,17.0,0.0,POINT (-11105561.614809224 2247611.0091226622),-1.110556e+07,2.247611e+06,1.0,334.266232
1,1,0.0,241101898.0,241101800.0,241101800.0,IT759157,GABRIEL T. VICENTE*,XAXX010101000,JOSE MARIA MORELOS SANTIAGO ACUTZILAPAN,NaN,...,19.779456,-99.759474,0241101898000000,17.0,0.0,POINT (-11105548.557845704 2247276.9980012453),-1.110555e+07,2.247277e+06,2.0,333.541286
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0000000000000nan,NaN,NaN,POINT (-11105470.475436851 2247932.7057738043),-1.110547e+07,2.247933e+06,3.0,330.736075
3,3,0.0,241101898.0,241101800.0,241101800.0,IT759157,GABRIEL T. VICENTE*,XAXX010101000,JOSE MARIA MORELOS SANTIAGO ACUTZILAPAN,NaN,...,19.779456,-99.759474,0241101898000000,17.0,0.0,POINT (-11105407.974873878 2246974.5311150667),-1.110541e+07,2.246975e+06,4.0,333.541286
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0000000000000nan,NaN,NaN,POINT (-11105245.6163556 2248175.2439046553),-1.110525e+07,2.248175e+06,5.0,226.941789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885218,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0000000000000nan,NaN,NaN,POINT (-11116503.791575344 2250017.7379621137),-1.111650e+07,2.250018e+06,71.0,161.477615
1885219,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0000000000000nan,NaN,NaN,POINT (-11116482.923019093 2250177.908433006),-1.111648e+07,2.250178e+06,72.0,161.492671
1885220,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0000000000000nan,NaN,NaN,POINT (-11115057.24816284 2252562.231224396),-1.111506e+07,2.252562e+06,1.0,1243.804891
1885221,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0000000000000nan,NaN,NaN,POINT (-11112238.74816284 2255004.9312243955),-1.111224e+07,2.255005e+06,4.0,1243.804891


In [ ]:
# ADS=NAUC.loc[NAUC['CLAVECATASTRAL'].isna()].drop_duplicates('geometry').dropna(how='all', axis=1)
ADS=ADS.to_crs(Manz.crs)

In [ ]:
ADS_RECTIFICADO=gpd.sjoin(ADS,Manz, how='left').drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

In [ ]:
m1=gpd.sjoin(casas_rectificado, ADS_RECTIFICADO)
ADS_F=ADS_RECTIFICADO.loc[~ADS_RECTIFICADO['LATLON'].isin(m1['LATLON'])]

In [ ]:
casas_f=casas_rectificado.loc[~casas_rectificado['ID'].isin(m1['ID'])]

In [ ]:
PRED=PRED.to_crs(Manz.crs)

In [ ]:
PRED_F=gpd.sjoin(PRED, casas_f)

In [ ]:
fin_casas=casas_f.loc[~casas_f['ID'].isin(PRED_F['ID'])]

In [ ]:
test0=pd.concat([m1,ADS_F,PRED_F, fin_casas], axis=0)
# .drop_duplicates('geometry')

In [ ]:
predial.loc[~predial['CLAVECATASTRAL'].isin(PRED['CLAVECATASTRAL'])]

### Distancias mínimas

In [ ]:
from shapely.geometry import Point, LineString
import geopandas as gpd
from geoloc2.preparacion_datos.src.preparacion_inter_puntos import ckdnearest
gdf=gpd.read_file("C:/Users/ASUS/Desktop/qgiszonas/points.shp")
gdf["LATLON"]=gdf["geometry"].x.astype("str")+","+gdf["geometry"].y.astype("str")
gdf


# usar primero sjoin_nearest y si hay mucho fallo usar la función ckdnearest

In [ ]:
casas=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\casas_t_especificios_1.shp")
casas=casas.to_crs(gdf.crs)
casas["LATLON"]=casas["geometry"].centroid.x.astype("str")+","+casas["geometry"].centroid.y.astype("str")
casas["geometry1"]=casas["geometry"]
casas["geometry"]=casas["geometry"].representative_point()
casas=casas.reset_index()
casas.rename(columns={"index":"FID"},inplace=True)
casas

In [ ]:
terreno=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\terreno_t_especificios_1.shp")
terreno=terreno.to_crs(gdf.crs)
terreno["LATLON"]=terreno["geometry"].centroid.x.astype("str")+","+terreno["geometry"].centroid.y.astype("str")
terreno["geometry1"]=terreno["geometry"]
terreno["geometry"]=terreno["geometry"].representative_point()
terreno

In [ ]:
JFF_c = ckdnearest(gdf.drop(columns='LATLON').to_crs(3857), casas.to_crs(3857))
# JFF_c = JFF_c.loc[JFF_c['min_dist'] < 10]
JFF_c

In [ ]:
JFF_terreno = ckdnearest(gdf.drop(columns='LATLON').to_crs(3857), terreno.to_crs(3857))
# JFF_terreno = JFF_terreno.loc[JFF_terreno['min_dist'] < 10]
JFF_terreno

In [ ]:
JFF_c["origen"]="Casas"
JFF_terreno["origen"]="Terreno"
concatenado=pd.concat([JFF_c,JFF_terreno],ignore_index=True)
concatenado.sort_values(by="min_dist",inplace=True)
concatenado=concatenado.drop_duplicates(subset=["CLAVECATAS"],keep="first")
concatenado

In [ ]:
concatenado[["FID","origen","CLAVECATAS","geometry"]].to_crs("3857").to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Puntos_cercanos.shp")


In [ ]:
a=concatenado[["FID","origen","CLAVECATAS","geometry1"]]
a=gpd.GeoDataFrame(a[["FID","origen","CLAVECATAS"]],geometry=a["geometry1"])
a.crs=6364
a=a.to_crs("3857")
a.to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Poligonos_cercanos.shp")